In [1]:
import pandas as pd
import time
import numpy as np
import os
# Load datasets from ml-latest-small folder
df_movies = pd.read_csv(os.path.join('ml-latest-small', 'movies.csv'))
df_ratings = pd.read_csv(os.path.join('ml-latest-small', 'ratings.csv'))
df_tags = pd.read_csv(os.path.join('ml-latest-small', 'tags.csv'))  


tags_grouped = df_tags.groupby('movieId')['tag'].apply(lambda x: ','.join(x)).reset_index()


# Merge tags with movies
movies_tag = pd.merge(df_movies, tags_grouped, on='movieId', how='left')
movies_tag['tag'] = movies_tag['tag'].fillna('')

 

movies_tag['combined_features'] = movies_tag['genres'] + ' ' + movies_tag['tag']

 

# Merge rate with movies_tag
movies_tag_rate = pd.merge(df_ratings,movies_tag,  on='movieId', how='left')

 

#calculate age in year and add a new column weighted_rating
movies_tag_rate['age_year']= round((round(time.time() )- movies_tag_rate['timestamp'])/(24*3600*365))
movies_tag_rate['weighted_rating'] = movies_tag_rate['rating'] *1/ movies_tag_rate['age_year']

 

 

 

# Calculate the mean of weighted_rating for each movie and sort it

movies_tag_rate.groupby("movieId").agg({

        "weighted_rating": "mean",

        "title": "first"

    }).sort_values(by="weighted_rating", ascending=False).reset_index()

,movieId,weighted_rating,title
0,172875,0.714286,A Detective Story (2003)
1,187717,0.714286,Won't You Be My Neighbor? (2018)
2,33649,0.714286,Saving Face (2004)
3,179135,0.714286,Blue Planet II (2017)
4,136556,0.714286,Kung Fu Panda: Secrets of the Masters (2011)
...,...,...,...
9719,5105,0.023810,Don't Look Now (1973)
9720,7742,0.023810,Baxter (1989)
9721,6967,0.023810,Dead of Night (1945)
9722,5700,0.022727,The Pumaman (1980)


In [27]:
import pandas as pd
import time
import numpy as np
import os
# Load datasets from ml-latest-small folder
df_movies = pd.read_csv(os.path.join('ml-latest-small', 'movies.csv'))
df_ratings = pd.read_csv(os.path.join('ml-latest-small', 'ratings.csv'))
df_tags = pd.read_csv(os.path.join('ml-latest-small', 'tags.csv'))  


tags_grouped = df_tags.groupby('movieId')['tag'].apply(lambda x: ','.join(x)).reset_index()


# Merge tags with movies
movies_tag = pd.merge(df_movies, tags_grouped, on='movieId', how='left')
movies_tag['tag'] = movies_tag['tag'].fillna('')

 

movies_tag['combined_features'] = movies_tag['genres'] + ' ' + movies_tag['tag']

 

# Merge rate with movies_tag
movies_tag_rate = pd.merge(df_ratings,movies_tag,  on='movieId', how='left')

 
# Calculate the count of rating for each movie 

weighted_movie_rate=movies_tag_rate.groupby(["movieId","title"]).agg({

        "rating": ["count" , "mean"]

    }).reset_index()
#.sort_values(by="rating", ascending=False).reset_index()
m=20;




def imbd_rating(movie_rate:pd.DataFrame, m=20)->pd.DataFrame:
    #score=v/(v+m).R +  m/(v+m).C
    #R: average
    #v: count
    R=movie_rate.groupby('movieId')['rating'].mean()
    v=movie_rate.groupby('movieId')['rating'].count()
    C=movie_rate['rating'].mean() 
    term1 = (v /(v +m))*R
    term2 = m/(v +m)*C
    return term1 + term2    

weighted_movie_rate['weighted_rating']=imbd_rating(movies_tag_rate,m=m)
weighted_movie_rate.sort_values(by=('weighted_rating', ''), ascending=False).reset_index()
weighted_movie_rate.head()

movieId                               title rating           weighted_rating
                                               count      mean                
0       1                    Toy Story (1995)    215  3.920930             NaN
1       2                      Jumanji (1995)    110  3.431818        3.885239
2       3             Grumpier Old Men (1995)     52  3.259615        3.442547
3       4            Waiting to Exhale (1995)      7  2.357143        3.326821
4       5  Father of the Bride Part II (1995)     49  3.071429        3.204857

In [ ]:
term1 = (weighted_movie_rate[('rating', 'count')] /(weighted_movie_rate[('rating', 'count')]+m))*weighted_movie_rate[('rating', 'mean')]
term2=m/(weighted_movie_rate[('rating', 'count')]+m)*C
weighted_movie_rate['weighted_rating'] = term1+term2
weighted_movie_rate.sort_values(by="weighted_rating", ascending=False).reset_index()